# Parte 2: Classificação de texto

> Implemente um exemplo de uso do BERT para uma tarefa de classificação de texto.
> Utilize qualquer conjunto de dados de sua escolha.

> Explique como transformaria isso em produto, seja específico.

In [ ]:
!pip install -r requirements.txt

## Explicação

Em prol da simplicidade da demonstração, utilizarei um modelo com BERT pelo spacy (en_core_web_trf). Para esse modelo, que é para processamentos em inglês, utilizarei dados em inglês.

### Ideia/Exemplo de produto

Um modelo que reconhece o pedido das pessoas, em produto e quantidade, permitindo que seja implementada uma automação de pedidos.

In [42]:
data = [
    "I'd like to have 10 apples.",
    "Twenty bags, please!",
    "Only one burguer for me.",
    "He is asking for 15 screws",
    "Could you bring me 3 bottles of water?",
    "She ordered seven pizzas.",
    "We'll need 25 plastic chairs.",
    "Give me two tickets to the concert.",
    "They bought 8 laptops.",
    "Just one slice of cake.",
]

In [43]:
import spacy

In [44]:
nlp = spacy.load("en_core_web_trf")

In [45]:
processed_data = [nlp(request) for request in data]
processed_data

[I'd like to have 10 apples.,
 Twenty bags, please!,
 Only one burguer for me.,
 He is asking for 15 screws,
 Could you bring me 3 bottles of water?,
 She ordered seven pizzas.,
 We'll need 25 plastic chairs.,
 Give me two tickets to the concert.,
 They bought 8 laptops.,
 Just one slice of cake.]

In [47]:
# Efetuo uma filtragem dos dados por sua classificação, para localizar a quantidade e produto que o cliente menciona
# Optei por utilizar uma lista de qty e prod dentro de args pois podem haver múltiplas ocorrências de nums. ou noun/propn.
# Pode ser feito um pós processamento após isso para determinar qual dos itens na lista é o de maior probabilidade de ser
# o desejado, ou fazer um join como eu fiz de forma a apresentar o nome mais completo dos produtos com múltiplas palavras
# (por exemplo "bottles of water", "slices of cake", "tickets for the concert")

filtered_data = []
for item in processed_data:
    args = {"qty": None, "prod": []}
    for token in item:
        if token.like_num:
            args["qty"] = token.text
        if token.pos_ in ("NOUN", "PROPN"):
            args["prod"].append(token.text)
    args["prod"] = " ".join(args["prod"])
    filtered_data.append(args)

filtered_data

[{'qty': '10', 'prod': 'apples'},
 {'qty': 'Twenty', 'prod': 'bags'},
 {'qty': 'one', 'prod': 'burguer'},
 {'qty': '15', 'prod': 'screws'},
 {'qty': '3', 'prod': 'bottles water'},
 {'qty': 'seven', 'prod': 'pizzas'},
 {'qty': '25', 'prod': 'chairs'},
 {'qty': 'two', 'prod': 'tickets concert'},
 {'qty': '8', 'prod': 'laptops'},
 {'qty': 'one', 'prod': 'slice cake'}]

In [48]:
# Transformando em um dataframe do pandas para melhor visualização
import pandas as pd

In [49]:
dict_for_dataframe = {
    "desc": data,
    "qty": [output["qty"] for output in filtered_data],
    "prod": [output["prod"] for output in filtered_data]
}

df = pd.DataFrame(dict_for_dataframe)

df

,desc,qty,prod
0,I'd like to have 10 apples.,10,apples
1,"Twenty bags, please!",Twenty,bags
2,Only one burguer for me.,one,burguer
3,He is asking for 15 screws,15,screws
4,Could you bring me 3 bottles of water?,3,bottles water
5,She ordered seven pizzas.,seven,pizzas
6,We'll need 25 plastic chairs.,25,chairs
7,Give me two tickets to the concert.,two,tickets concert
8,They bought 8 laptops.,8,laptops
9,Just one slice of cake.,one,slice cake


Essa demonstração mais simples também exemplifica como os modelos, mesmo com boas taxas de acerto, necessitam de refinamento e treinamento para retornar dados mais próximos do desejado.